In [ ]:
pip install transformers
pip install emoji

In [ ]:
pip install emoji
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 3.3 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

max_sequence_length = 514

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

data = 'semstopwords.csv'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def encode_labels(data, label_column):
    # Cria um dicionário de mapeamento para os rótulos
    label_map = {"positive": 1, "negative": 0}

    # Aplica a função de mapeamento para criar uma nova coluna com os valores codificados
    data['encoded_labels'] = data[label_column].map(label_map)

    return data


def tokenize_with_roberta(csv_file, column_name, max_length=128):
    data = pd.read_csv(csv_file)
    data.head()
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    tokenized = data[column_name].apply(lambda x: tokenizer.encode(x, max_length=max_length,padding='max_length', truncation=True))
    data['tokens'] = tokenized
    return data


tokenized_data = tokenize_with_roberta(data,'review',max_length = 512)
tokenized_data = encode_labels(tokenized_data,"sentiment")
tokenized_data.head()

,review,sentiment,tokens,encoded_labels
0,one of the other reviewers has mentioned that ...,positive,"[0, 1264, 9, 5, 97, 34910, 34, 2801, 14, 71, 2...",1
1,wonderful little production . < br / > < br / ...,positive,"[0, 605, 17037, 2650, 410, 931, 479, 28696, 53...",1
2,i thought this was wonderful way to spend time...,positive,"[0, 118, 802, 42, 21, 4613, 169, 7, 1930, 86, ...",1
3,basically there 's family where little boy ( j...,negative,"[0, 15609, 3435, 89, 128, 29, 284, 147, 410, 2...",0
4,petter mattei 's `` love in the time of money ...,positive,"[0, 13713, 1334, 34678, 118, 128, 29, 45518, 6...",1


In [ ]:
type(tokenized_data["tokens"][0][0])

int

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import AdamW
from torch.utils.data import DataLoader, TensorDataset

train_data, test_data = train_test_split(tokenized_data, test_size=0.2)

train_inputs = torch.tensor(train_data['tokens'].tolist())
train_labels = torch.tensor(train_data['encoded_labels'].tolist())

test_inputs = torch.tensor(test_data['tokens'].tolist())
test_labels = torch.tensor(test_data['encoded_labels'].tolist())

# Crie datasets do PyTorch
train_dataset = TensorDataset(train_inputs, train_labels)
test_dataset = TensorDataset(test_inputs, test_labels)

# Crie dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

# Carregue o modelo pre-treinado para fine-tuning
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Configure o otimizador
optimizer = AdamW(model.parameters(), lr=1e-5)

# Treinamento
epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Avaliação no conjunto de teste
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs, labels = batch
        outputs = model(inputs)
        _, predicted = torch.max(outputs.logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Acurácia no conjunto de teste: {accuracy}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



# Dividindo os dados em features e rótulos
X = np.array(tokenized_data['tokens'].tolist())
y = np.array(tokenized_data['encoded_labels'])  # Substitua 'rotulo' pelo nome da coluna de rótulos

# Dividindo em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertendo para objetos DMatrix do XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Definindo parâmetros do modelo
params = {
    'max_depth': 10,
    'learning_rate': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss'
}

# Treinando o modelo
num_round = 100
model = xgb.train(params, dtrain, num_round)

# Fazendo previsões no conjunto de teste
predictions_proba = model.predict(dtest)
predictions = [1 if pred > 0.5 else 0 for pred in predictions_proba]

# Calculando a acurácia do modelo
accuracy = accuracy_score(y_test, predictions)
print(f'Acurácia do modelo: {accuracy * 100:.2f}%')

In [ ]:
# Roberta, Imdb_lemmatization.csv: 0.9504
# Xgboost,  Imdb_lemmatization.csv: 0.5620

# Roberta, Imdb_stemming.csv: 0.9448
# Xgboost,  Imdb_stemming.csv: 0.5403

# Roberta, Imbd_steeming_and_lemmatization.csv: 0.9427
# Xgboost,  Imbd_steeming_and_lemmatization.csv:: 0,5498

